In [5]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
%autoreload 2

In [7]:
from pynq.overlays.h264 import H264Overlay
from pynq.lib.video import *

base = H264Overlay("h264.bit")
hdmi_in = base.video.hdmi_in

load ready


In [8]:
from pynq import MMIO
rgb2yuv = MMIO(base.ip_dict['h264/rgb2yuv_with_axi_0']['phys_addr'], 0x10000)
h264 = MMIO(base.ip_dict['h264/h264enc_with_axi_0']['phys_addr'], 0x10000)

from h264py.h264 import H264
h264_send = H264()

from pynq import Xlnk
xlnk = Xlnk()

In [12]:
size = 1280*720*4

## repeat from here

In [14]:
with hdmi_in.configure(pixelformat=PIXEL_RGBA):
    hdmi_in.start()
    cma_recv = xlnk.cma_array((size,), dtype=np.uint8)
    result = xlnk.cma_array((size,), dtype=np.uint8)
    start = time.time()
    for i in range(1000):
        
        print(i)
        
        cma_send = hdmi_in.readframe()
        
        rgb2yuv.write(0x04, cma_send.physical_address)
        cma_send.tofile('pic.rgb')
        rgb2yuv.write(0x08, cma_recv.physical_address)
        cma_recv.tofile('pic.yuv')
        rgb2yuv.write(0x0c, 720)
        rgb2yuv.write(0x10, 1280)
        rgb2yuv.write(0x14, 1280*720)
        rgb2yuv.write(0x00, 1)
        rgb2yuv.write(0x00, 0)
        while rgb2yuv.read(0x18)==1: pass
        while rgb2yuv.read(0x18)==0: pass
        
        h264.write(0x04, 0x00002C4F) # length 1280*720
        h264.write(0x08, 0)
        h264.write(0x14, cma_recv.physical_address)
        h264.write(0x18, result.physical_address)
        h264.write(0x00, 1)
        while h264.read(0x24)==1: pass
        while h264.read(0x24)==0: pass

        h264_size = h264.read(0x1c)
        h264_send.send_frame(result[:h264_size])

        cma_send.freebuffer()
        
    print(time.time()-start)

cma_recv.freebuffer()
result.freebuffer() 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120


KeyboardInterrupt: 

In [ ]:
xlnk.xlnk_reset()